In [1]:
import pandas as pd

# Get matches data

# Have to manually correct the CSV file. On line 955, there is a cell where there is a starting "
# but without the closing ". Need to delete the " mark.

#use xls, not .csv or .xlsx, these usually have errors
df_matches = pd.read_excel("charting-m-matches_clean.xls")

In [2]:
# There are several duplicated matches. Remove them. 
df_matches.drop_duplicates(subset="match_id", inplace=True)

# Data cleaning
df_matches["Player 1"] = df_matches["Player 1"].str.strip()
df_matches["Player 2"] = df_matches["Player 2"].str.strip()
df_matches["Player 1"] = df_matches["Player 1"].replace(to_replace=r"\s+", value="_", regex=True)
df_matches["Player 2"] = df_matches["Player 2"].replace(to_replace=r"\s+", value="_", regex=True)
df_matches["Player 1"] = df_matches["Player 1"].str.lower()
df_matches["Player 2"] = df_matches["Player 2"].str.lower()
df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].str.strip()
df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].str.strip()
df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].str.lower()
df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].str.lower()
df_matches["Tournament"] = df_matches["Tournament"].str.strip()
df_matches["Tournament"] = df_matches["Tournament"].str.lower()
df_matches["Tournament"] = df_matches["Tournament"].replace(to_replace=r"\s+", value="_", regex=True)
df_matches["Surface"] = df_matches["Surface"].str.strip()
df_matches["Surface"] = df_matches["Surface"].str.lower()

df_matches = df_matches.astype({"Date": "str"})
# df_matches = df_matches.astype({"Best of": "int32"})

df_matches["Surface"] = df_matches["Surface"].fillna("unknown")
df_matches["Tournament"] = df_matches["Tournament"].fillna("unknown")
df_matches["Best of"] = df_matches["Best of"].fillna(0)
df_matches["Player 1"] = df_matches["Player 1"].fillna("unknown")
df_matches["Player 2"] = df_matches["Player 2"].fillna("unknown")
df_matches["Pl 1 hand"] = df_matches["Pl 1 hand"].fillna("unknown")
df_matches["Pl 2 hand"] = df_matches["Pl 2 hand"].fillna("unknown")
df_matches["Round"] = df_matches["Round"].fillna("unknown")

df_matches.info()
print()

print("There are %d matches in the database." % len(df_matches))
# df_matches.head(30)

df_matches.to_excel("charting-m-matches-cleaned.xls")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2573 entries, 0 to 2575
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   match_id    2573 non-null   object
 1   Player 1    2573 non-null   object
 2   Player 2    2573 non-null   object
 3   Pl 1 hand   2573 non-null   object
 4   Pl 2 hand   2573 non-null   object
 5   Gender      2573 non-null   object
 6   Date        2573 non-null   object
 7   Tournament  2573 non-null   object
 8   Round       2573 non-null   object
 9   Time        1495 non-null   object
 10  Court       1929 non-null   object
 11  Surface     2573 non-null   object
 12  Umpire      1594 non-null   object
 13  Best of     2573 non-null   int64 
 14  Final TB?   2566 non-null   object
 15  Charted by  2573 non-null   object
dtypes: int64(1), object(15)
memory usage: 341.7+ KB

There are 2573 matches in the database.


In [3]:
df_matches["Player 1"].value_counts(dropna=False)

roger_federer            298
novak_djokovic           202
rafael_nadal             100
pete_sampras              85
andy_murray               61
                        ... 
ramkumar_ramanathan        1
carlos_alcaraz_garfia      1
milos_sekulic              1
alexey_vatutin             1
taylor_dent                1
Name: Player 1, Length: 418, dtype: int64

In [4]:
# How many matches for each player in the database?
players = df_matches["Player 1"].append(df_matches["Player 2"])
player_match_count = players.value_counts()

print("Match count per player in the database.")
player_match_count.head(20)

Match count per player in the database.


roger_federer            448
novak_djokovic           275
rafael_nadal             267
andy_murray              153
stefan_edberg            130
pete_sampras             109
andre_agassi             104
lleyton_hewitt            97
juan_martin_del_potro     84
stanislas_wawrinka        70
boris_becker              65
ivan_lendl                65
gael_monfils              56
david_ferrer              53
andy_roddick              51
alexander_zverev          51
dominic_thiem             51
tomas_berdych             50
milos_raonic              47
karim_mohamed_maamoun     46
dtype: int64

In [5]:
# Test handedness
# print(df_matches.loc[df_matches["Player 1"] == "corentin_moutet"]["Pl 1 hand"].unique())
# print(df_matches.loc[df_matches["Player 2"] == "corentin_moutet"]["Pl 2 hand"].unique())

In [6]:
# Get a list of unique players
unique_players = players.unique()
# type(unique_players)

print("There are %d players in the database." % len(unique_players))

There are 549 players in the database.


In [7]:
# clean the handedness columns because there are errors. Create a table of players and their handednesses.
df_players_handedness = pd.DataFrame()

for player in unique_players:
    
    #these two lines select the columns "Pl 1 hand" and "Pl 2 hand" from df_matches for each player
    selected_player1_hand = df_matches[(df_matches["Player 1"] == player)]["Pl 1 hand"]
    selected_player2_hand = df_matches[(df_matches["Player 2"] == player)]["Pl 2 hand"]
    
    #correct_handedness needs to have an initial value
    #so that if there is no error in handedness,
    #we can just use the given handedness
    #if there is an error, it will display later in the if function
    if len(selected_player1_hand) > 0:
        # Use the majority of the 
        correct_handedness = selected_player1_hand.value_counts().idxmax()
    elif len(selected_player2_hand) > 0:
        correct_handedness = selected_player2_hand.value_counts().idxmax()
    else:
        correct_handedness = None
    
    #assign variable so that yoou do not have to continuously use the value_counts() function
    selected_player_hand_value_counts = selected_player1_hand.value_counts()
    
    #if the player has more than one handedness in Pl 1 hand column,
    #replace the wrong hand with correct hand
    if (len(selected_player_hand_value_counts) > 1):
        print(player)
        print(selected_player_hand_value_counts)
        
        #finding which hand occurs more frequently for the player
        #this is also the correct handedness for that player
        correct_handedness = selected_player_hand_value_counts.idxmax()
        
        #finding which hand occurs less frequently for the player
        #this is also the wrong handedness for that player
        wrong_handedness = selected_player_hand_value_counts.idxmin()
        
        #must save back to the original column and replace wrong_hand with correct_hand
        df_matches.loc[(df_matches["Player 1"] == player), "Pl 1 hand"] = \
        df_matches.loc[(df_matches["Player 1"] == player), "Pl 1 hand"].replace(wrong_handedness, correct_handedness)
        
    #same as above, only for the "Pl 2 hand" column
    selected_player_hand_value_counts = selected_player2_hand.value_counts()
    if (len(selected_player_hand_value_counts) > 1):
        print(player)
        print(selected_player_hand_value_counts)
        correct_handedness = selected_player_hand_value_counts.idxmax()
        wrong_handedness = selected_player_hand_value_counts.idxmin()
        df_matches.loc[(df_matches["Player 2"] == player), "Pl 2 hand"] = \
        df_matches.loc[(df_matches["Player 2"] == player), "Pl 2 hand"].replace(wrong_handedness, correct_handedness)
    
    row = {"player": player, "handedness": correct_handedness}
    df_players_handedness = df_players_handedness.append(row, ignore_index=True)
#     print(player)

# df_players_handedness now has the correct handedness for each player. 
print(df_players_handedness)

stanislas_wawrinka
r    27
l     1
Name: Pl 2 hand, dtype: int64
rafael_nadal
l    99
r     1
Name: Pl 1 hand, dtype: int64
rafael_nadal
l    166
r      1
Name: Pl 2 hand, dtype: int64
kyle_edmund
r    3
l    1
Name: Pl 2 hand, dtype: int64
nick_kyrgios
r    9
l    1
Name: Pl 1 hand, dtype: int64
thomas_muster
l    11
r     1
Name: Pl 2 hand, dtype: int64
    handedness                     player
0            r             ernests_gulbis
1            r           alexander_bublik
2            r          mikhail_kukushkin
3            r             novak_djokovic
4            r                marin_cilic
..         ...                        ...
544          r  christophe_roger_vasselin
545          r              brian_teacher
546          r                jiri_hrebec
547          r            adriano_panatta
548          r               ken_rosewall

[549 rows x 2 columns]


In [8]:
# Save the results to XLS files for use later.

df_players_handedness.dropna(inplace=True)
df_players_handedness.to_excel("player-handedness.xls")

In [9]:
# Find out how many right handed and left handed players are in the database

unique_handed_players = df_players_handedness["handedness"]
unique_handed_players_value_counts = unique_handed_players.value_counts()
unique_handed_players_value_counts.isnull().values.any()
print(len(unique_handed_players))
print("There are " + str(unique_handed_players_value_counts["r"]) + " right-handed players and " + 
      str(unique_handed_players_value_counts["l"]) + " left-handed players.")
right_handed_count = 0
left_handed_count = 0
other_count = 0
for index,value in unique_handed_players.items():
    #must represent with quotation marks
    #to indicate it is a string
    if value == "r":
        right_handed_count += 1
    elif value == "l":
        left_handed_count += 1
    else:
        print(value)
        other_count += 1
        
print("There are " + str(right_handed_count) + " right-handed players and " + 
      str(left_handed_count) + " left-handed players, with " + str(other_count) + " oddball handedness.")

549
There are 471 right-handed players and 78 left-handed players.
There are 471 right-handed players and 78 left-handed players, with 0 oddball handedness.


In [10]:
# Find the number of matches per tournament
tournaments = df_matches["Tournament"]
tournaments_value_counts = tournaments.value_counts(dropna=False)
print(tournaments_value_counts.head(40))

australian_open            210
wimbledon                  190
us_open                    175
roland_garros              126
indian_wells_masters       121
tour_finals                112
miami_masters               93
rome_masters                70
cincinnati_masters          69
monte_carlo_masters         65
paris_masters               64
canada_masters              61
madrid_masters              51
shanghai_masters            50
dubai                       39
rotterdam                   34
doha                        32
queens_club                 31
halle                       30
basel                       29
olympics                    29
brisbane                    27
masters_cup                 25
washington                  25
hamburg_masters             24
barcelona                   20
beijing                     18
sydney                      18
tokyo                       17
stuttgart                   17
marseille                   16
rio_de_janeiro              16
stockhol

In [11]:
#prints how many times roger_federer appears in Player 1
len(df_matches.loc[(((df_matches["Player 1"] == "roger_federer") & (df_matches["Player 2"] == "rafael_nadal")) | 
                  ((df_matches["Player 1"] == "rafael_nadal") & (df_matches["Player 2"] == "roger_federer")))])


35

In [12]:
# Select matches based on different conditions
def select_matches(data, player1=None, player2=None, tournament=None, surface=None, 
                   player1_handedness=None, player2_handedness=None, best_of=None, date=None):
    df_results = data.copy()
    
    if player1 != None:
        df_results = df_results.loc[(df_results["Player 1"].str.contains(player1)) | (df_results["Player 2"].str.contains(player1))]
    
    if player2 != None:
        df_results = df_results.loc[(df_results["Player 1"].str.contains(player2)) | (df_results["Player 2"].str.contains(player2))]
        
    if tournament != None:
        df_results = df_results.loc[(df_results["Tournament"].str.contains(tournament))]
        
    if surface != None:
        df_results = df_results.loc[(df_results["Surface"].str.contains(surface))]
    
    if (player1 != None) & (player2_handedness != None):
        df_results = df_results.loc[((df_results["Player 1"].str.contains(player1)) & (df_results["Pl 2 hand"] == player2_handedness)) | 
                                   ((df_results["Player 2"].str.contains(player1)) & (df_results["Pl 1 hand"] == player2_handedness))]
    
    if (player2 != None) & (player1_handedness != None):
        df_results = df_results.loc[((df_results["Player 1"].str.contains(player2)) & (df_results["Pl 2 hand"] == player1_handedness)) | 
                                   ((df_results["Player 2"].str.contains(player2)) & (df_results["Pl 1 hand"] == player1_handedness))]
        
    if (best_of != None):
        df_results = df_results.loc[(df_results["Best of"] == best_of)]
        
    if (date != None):
        df_results = df_results.loc[(df_results["Date"].str.contains("^" + date))]
                                    
    return df_results
    

In [13]:
# Find the number of matches for a player
player_name = "nadal"

len(select_matches(df_matches, player1=player_name))

267

In [14]:
# Find the head-to-head match count for two players
player1_name = "nadal"
player2_name = "federer"

len(select_matches(df_matches, player1=player1_name, player2=player2_name))

35

In [229]:
# Find the number of head-to-head match count for any two players in the database



## Analyzing Point-by-Point statistics

In [230]:
#when you download very large csv files and the files can't be read,
#you can use excel to save it as "CSV UTF-8(Comma delimited)" format
#then you can read the entire spreadsheet
df_points = pd.read_csv("charting-m-points_1.csv", low_memory=False)

df_points = df_points.astype({"rallyLen": "float64"})
df_points["Pts"] = df_points["Pts"].str.replace("Jan", "1", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Feb", "2", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Mar", "3", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Apr", "4", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("May", "5", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Jun", "6", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Jul", "7", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Aug", "8", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Sep", "9", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Oct", "10", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Nov", "11", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("Dec", "12", case=False)
df_points["Pts"] = df_points["Pts"].str.replace("00", "0", case=False)

df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Jan", "1", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Feb", "2", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Mar", "3", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Apr", "4", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("May", "5", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Jun", "6", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Jul", "7", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Aug", "8", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Sep", "9", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Oct", "10", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Nov", "11", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("Dec", "12", case=False)
df_points["PtsAfter"] = df_points["PtsAfter"].str.replace("00", "0", case=False)

KeyboardInterrupt: 

In [186]:
#the purpose is to extract the digit in the parentheses
#but just doing it straight up causes ambiguity, so I need to add a \ in front of the parentheses
#to indicate that this parentheses is just a character, not a special symbol (capture group)
# The following code does not work because in some matches, "Gm#" only has game index, not point index. 
# df_points_selected["serve_side"] = df_points_selected["Gm#"].str.extract(r".\((\d+)\)").astype("int64")
# df_points_selected["serve_side"] = df_points_selected["serve_side"] % 2
# df_points_selected["serve_side"].replace({0: "ad", 1: "deuce"}, inplace=True)

# Identify serve side for each point

import re

def identify_serve_side(data):
    data = data.copy()
    # List all the possible scores (before serve) and their corresponding serve sides. 
    dict_serve_side = {"0-0": "deuce", 
                       "0-15": "ad", 
                      "15-0": "ad",
                      "15-15": "deuce",
                      "30-0": "deuce",
                      "0-30": "deuce",
                      "30-15": "ad",
                      "15-30": "ad",
                      "40-0": "ad",
                      "0-40": "ad",
                      "40-15": "deuce",
                      "15-40": "deuce",
                       "30-30": "deuce",
                       "40-30": "ad",
                       "30-40": "ad",
                       "40-40": "deuce",
                       "40-AD": "ad",
                       "AD-40": "ad"
                      }

    data["serve_side"] = None

    for index, row in data.iterrows():
        if data.loc[index, "TB?"] == "0":
            # Identify serve sides based on the score (before serve)
            data.loc[index, "serve_side"] = dict_serve_side[data.loc[index, "Pts"]]
        elif data.loc[index, "TB?"] == "1":
            # For tiebreak points, if the sum of the two scores (before serve) are even, it's on the deuce side.
            # If the sum of the two scores (before serve) are odd, it's on the ad side. 
    #         print(data.loc[index, "Pts"])

            # Retrieve the first score
            tb_point1_str = re.search("^(\d+)-", data.loc[index, "Pts"])
            if tb_point1_str:
                tb_point1 = int(tb_point1_str.group(1))
            # Retrieve the second score
            tb_point2_str = re.search("-(\d+)", data.loc[index, "Pts"])
            if tb_point2_str:
                tb_point2 = int(tb_point2_str.group(1))

            if ((tb_point1 + tb_point2) % 2) == 0:
                data.loc[index, "serve_side"] = "deuce"
            elif ((tb_point1 + tb_point2) % 2) == 1:
                data.loc[index, "serve_side"] = "ad"

    # data[["Pts", "TB?", "serve_side"]]
    return data

In [187]:
# Separate serve direction from serve outcome
# Need to identify +

def identify_serve_direction_outcome(data):
    data = data.copy()
    
    data["Sv1_direction"] = data["Sv1"].str.extract(r"^(\d)")
    data.loc[data["Sv1_direction"].isin(["4", "5", "6"]) == False, "Sv1_direction"] = "0"
    data["Sv1_direction"].fillna(value="0", inplace=True)
    # Replace numeric code with a word. May need to keep the numbers for stats analysis. 
    data["Sv1_direction"].replace({"4": "wide", "5": "body", "6": "t", "0": "unknown"}, inplace=True)

    #whatever is inside the parentheses is what is being captured/retrieved
    data["Sv1_outcome"] = data["Sv1"].str.extract(r"^\d(.+)")
    data["Sv1_outcome"].replace({"n": "net", "d": "deep", "*": "ace", "w": "wide", "#": "unreturnable", 
                                               "x": "wide_and_deep", "+": "serve_and_volley_"}, inplace=True)
    # To-do: replace na with "unknown". Same for below. 

    data["Sv2_direction"] = data["Sv2"].str.extract(r"^(\d)")
    data.loc[data["Sv2_direction"].isin(["4", "5", "6"]) == False, "Sv2_direction"] = "0"
    data["Sv2_direction"].fillna(value="0", inplace=True)
    data["Sv2_direction"].replace({"4": "wide", "5": "body", "6": "t", "0": "unknown"}, inplace=True)

    data["Sv2_outcome"] = data["Sv2"].str.extract(r"^\d(.+)")
    data["Sv2_outcome"].replace({"n": "net", "d": "deep", "*": "ace", "w": "wide", "#": "unreturnable", 
                                               "x": "wide_and_deep", "+": "serve_and_volley_"}, inplace=True)

    # data[["Pts", "serve_side", "Sv1", "Sv2", "Sv1_direction", "Sv2_direction", "Sv1_outcome", "Sv2_outcome"]]

    # print(data["Sv1_outcome"].value_counts())
    # data.info()
    return data

In [189]:
# Identify server name
def identify_server_name(data):
    data = data.copy()
    
    data["server_name"] = None
    data["returner_name"] = None

    for index, row in data.iterrows():
        # to-do: Let user pass df_matches
        # This returns a dataframe. 
        players = df_matches.loc[df_matches["match_id"] == row["match_id"], ["Player 1", "Player 2"]]
        
        # Sometimes there may be no match for a match id
        if players.empty != True:
            if(len(players) > 1): 
                print(row["match_id"] + " error: The same match_id appears more than once in df_matches.")
                break

            # Must reset index. Sometimes the index is not 0. 
            players = players.reset_index(drop=True)
#             print(players)
            player1 = players.at[0, "Player 1"]
            player2 = players.at[0, "Player 2"]
            
            if player1 == None:
                print(row["match_id"] + ": player 1 is nan")
            elif player2 == None:
                print(row["match_id"] + ": player 2 is nan")

            if row["Svr"] == 1:
                # Save data directly to the data frame.
                # Do not save data to row["Svr"] because it will not be saved to the data frame.
                # Note that my_match is a data frame, not a series, even through there is only one row. 
                # So we must retrieve the first row from my_match. 
                data.at[index, "server_name"] = player1
                data.at[index, "returner_name"] = player2
            elif row["Svr"] == 2:
                data.at[index, "server_name"] = player2
                data.at[index, "returner_name"] = player1
        else:
            print(row["match_id"] + ": has no player?")
    # data[["server_name", "Svr", "Serving", "Pts", "serve_side", "Sv1", "Sv2", "Sv1_direction", "Sv2_direction", "Sv1_outcome", "Sv2_outcome"]]

    # data.to_csv("fed_nadal_points.csv")
    return data

In [190]:

%time df_points = identify_serve_side(df_points)
print("finished identifying serve side")                    

%time df_points = identify_serve_direction_outcome(df_points)
print("finished identifying serve direction and outcome")

%time df_points = identify_server_name(df_points)
print("finished identifying serve direction and outcome")

df_points.to_csv("charting-m-points_with_serve_side.csv")

Wall time: 40min 24s
finished identifying serv side
Wall time: 1.9 s
finished identifying serve direction and outcome
Wall time: 9min 3s
finished identifying serve direction and outcome


In [15]:
df_points = pd.read_csv("charting-m-points_with_serve_side.csv", low_memory=False)

In [16]:
def select_players_points(point_data, match_data):

    # Use a copy to avoid "A value is trying to be set on a copy of a slice from a DataFrame" warning. 
    df_points_selected = point_data.loc[point_data["match_id"].isin(match_data["match_id"])].copy()

    # df_points_selected.info()
    return df_points_selected

In [17]:
# import seaborn as sns

from scipy.stats import chi2_contingency
from scipy.stats import chi2

# Chi-square independence test
def chi_square(data, print_table = False):

    table = data.copy()

#     table = table.set_index("server")

    test_stats, p, dof, expected = chi2_contingency(table)
    print("chi-squared test results:")
    print("chi2 test stats = %f" % test_stats)
    print("p = %f" % p)
    print('dof = %d ' % dof)

     # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob,
    critical, test_stats))
    if abs(test_stats) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')

     # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')

    if print_table:
        print(table)

In [18]:
def compare_players_chi_square(table):
    servers = table["server"].unique()
    serve_sides = table["serve_side"].unique()
    serve_sequences = table["serve_sequence"].unique()
    
    for serve_side in serve_sides:
        for serve_sequence in serve_sequences:
            my_data = table.loc[(table["serve_side"] == serve_side) & (table["serve_sequence"] == serve_sequence), 
                               ["server", "wide", "body", "t"]]
            
            print("serve_side: " + serve_side)
            print(serve_sequence)
            chi_square(data=my_data, print_table=True)
            print()

In [19]:
from scipy.stats import chisquare
def serve_direction_test_of_even_distribution(point_data, match_data, player, opponent=None):
    surfaces = ["hard", "clay", "grass"]
    serve_sides = ["ad", "deuce"]
    
    serve_sequences = ["Sv1_direction", "Sv2_direction"]
        
    contingency_table = pd.DataFrame()
    
    for serve_sequence in serve_sequences:
        print(serve_sequence)
        for serve_side in serve_sides:
            print(serve_side)
            for surface in surfaces:
                print(surface)
                matches_by_surface = select_matches(data=match_data, player1=player, player2=opponent, surface=surface)
                selected_points = select_players_points(point_data, matches_by_surface)
                print("finished selecting points")

                if selected_points.empty != True:
                    my_data = selected_points.loc[(selected_points["server_name"].str.contains(player))]
                    # Without Series, you cannot use value_counts()
                    serve_frequency = my_data.loc[my_data["serve_side"] == serve_side, serve_sequence].value_counts()
                    serve_frequency.drop(labels="unknown", errors="ignore", inplace=True)
                    # Replace nan with 0
#                     serve_frequency.fillna(value=0, inplace=True)
                    # Sometimes the serve direction is not entered and is marked as nan, remove these counts. 
                    
                    serve_frequency_list = serve_frequency.tolist()
                    print(serve_frequency)
                    chisq, p = chisquare(serve_frequency_list)
                    print("chi-square: " + str(chisq))
                    print("p: " + str(p))
                    if p < 0.05:
                        print("Reject null hypothesis, not evenly distributed.\n")
                    else:
                        print("Cannot reject null hypothesis, possibly evenly distributed.\n")
                else:
                    print("No matches found")


In [20]:
#some players may generate errors due to NaN existing in the "Surface" column
serve_direction_test_of_even_distribution(point_data=df_points, match_data=df_matches, player="nadal", opponent=None)
# nadal_matches = select_matches(data=df_matches, player1="nadal")
# nadal_matches["Surface"].value_counts(dropna=False)
# possible even distribution:
# goffin, shapovalov, tsitsipas, wawrinka, pouille

Sv1_direction
ad
hard
finished selecting points
wide    2572
t       1469
body    1061
Name: Sv1_direction, dtype: int64
chi-square: 718.5797726381811
p: 9.170330077396573e-157
Reject null hypothesis, not evenly distributed.

clay
finished selecting points
wide    2460
t        837
body     732
Name: Sv1_direction, dtype: int64
chi-square: 1397.651526433358
p: 3.1902684074094687e-304
Reject null hypothesis, not evenly distributed.

grass
finished selecting points
wide    611
t       585
body    216
Name: Sv1_direction, dtype: int64
chi-square: 207.40934844192634
p: 9.15444960546219e-46
Reject null hypothesis, not evenly distributed.

deuce
hard
finished selecting points
t       2842
wide    1759
body    1012
Name: Sv1_direction, dtype: int64
chi-square: 905.0058792089792
p: 3.023223708886705e-197
Reject null hypothesis, not evenly distributed.

clay
finished selecting points
t       2172
body    1238
wide    1010
Name: Sv1_direction, dtype: int64
chi-square: 514.6117647058824
p: 1.7925

In [ ]:
# Check if the serve direction patterns vary from match to match on the same surface and same serve side (use MANOVA)

In [21]:
# Check if there is a difference in serve directions between ad and deuce sides
def compare_players_serves_on_different_sides(player, first_serve=True, surface=None, opponent=None):
    selected_matches = select_matches(data=df_matches, player1=player, player2=opponent, surface=surface)
    selected_points = select_players_points(df_points, selected_matches)
    
    if first_serve == True:
        serve_sequence = "Sv1_direction"
    else:
        serve_sequence = "Sv2_direction"
        
    if selected_points.empty != True:
        my_data = selected_points.loc[(selected_points["server_name"].str.contains(player))]
        contingency_table = pd.DataFrame()
            
        for serve_side in my_data["serve_side"].unique():
            # Must use .loc[], as not using .loc[] does not result in a Series
            #Without Series, you cannot use value_counts()
            serve_frequency = my_data.loc[(my_data["serve_side"] == serve_side), serve_sequence].value_counts()
            serve_frequency.drop(labels="unknown", errors="ignore", inplace=True)
            serve_frequency["serve_side"] = serve_side
            contingency_table = contingency_table.append(serve_frequency, ignore_index = True)
        contingency_table = contingency_table.dropna(how="all")
        contingency_table = contingency_table.set_index("serve_side")
#         contingency_table.fillna(value=0, inplace=True)
        contingency_table = contingency_table[["wide", "body", "t"]]
        print(contingency_table)
        chi_square(contingency_table, print_table = False)
    else:
        print("No matches found")

In [22]:
compare_players_serves_on_different_sides("cuevas", first_serve=True, surface="clay", opponent=None)

            wide  body     t
serve_side                  
deuce       46.0  14.0  93.0
ad          96.0  14.0  32.0
chi-squared test results:
chi2 test stats = 47.028853
p = 0.000000
dof = 2 
probability=0.950, critical=5.991, stat=47.029
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)


In [23]:
# Check if there is a difference in serve directions between ad and deuce sides
def compare_players_serves_on_different_surfaces(player, first_serve=True, opponent=None):
    surfaces = ["hard", "clay", "grass"]
    serve_sides = ["ad", "deuce"]
    
    if first_serve == True:
        serve_sequence = "Sv1_direction"
    else:
        serve_sequence = "Sv2_direction"
        
    contingency_table = pd.DataFrame()
    
    for serve_side in serve_sides:
        print(serve_side)
        for surface in surfaces:
            matches_by_surface = select_matches(data=df_matches, player1=player, player2=opponent, surface=surface)
            selected_points = select_players_points(df_points, matches_by_surface)

            if selected_points.empty != True:
                my_data = selected_points.loc[(selected_points["server_name"].str.contains(player))]

                # Without Series, you cannot use value_counts()
                serve_frequency = my_data.loc[my_data["serve_side"] == serve_side, serve_sequence].value_counts()
                serve_frequency.drop(labels="unknown", errors="ignore", inplace=True)
                serve_frequency["surface"] = surface
                contingency_table = contingency_table.append(serve_frequency, ignore_index = True)    
            else:
                print("No matches found")

        contingency_table = contingency_table.dropna(how="all")
        contingency_table = contingency_table.set_index("surface")
        contingency_table.fillna(value=0, inplace=True)
        contingency_table = contingency_table[["wide", "body", "t"]]
        print(contingency_table)
        chi_square(contingency_table, print_table = False)
        contingency_table.drop(contingency_table.index, inplace=True)

In [24]:
compare_players_serves_on_different_surfaces("murray", first_serve=True)

ad
           wide   body       t
surface                       
hard     1981.0  409.0  1992.0
clay      299.0   62.0   305.0
grass     447.0   64.0   426.0
chi-squared test results:
chi2 test stats = 6.671878
p = 0.154278
dof = 4 
probability=0.950, critical=9.488, stat=6.672
Independent (fail to reject H0)
significance=0.050, p=0.154
Independent (fail to reject H0)
deuce
           wide   body       t
surface                       
hard     2272.0  434.0  2104.0
clay      355.0   69.0   307.0
grass     426.0   54.0   562.0
chi-squared test results:
chi2 test stats = 45.683409
p = 0.000000
dof = 4 
probability=0.950, critical=9.488, stat=45.683
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)


In [25]:
def compare_player_serves_when_they_play_each_other(point_data, match_data, player1, player2, surface):
    
    my_matches = select_matches(data=match_data, player1=player1, player2=player2, surface=surface)
    df_points_selected = select_players_points(point_data=point_data, match_data=my_matches)
    
    if df_points_selected.empty == False:      
        players = [player1, player2]
        serve_sides = ["deuce", "ad"]
        serve_seqs = ["Sv1_direction", "Sv2_direction"]
        surfaces = ["hard", "clay", "grass"]
        
        print(surface)
        for serve_side in serve_sides:
            print(serve_side)
            for serve_seq in serve_seqs:
                print(serve_seq)
                contingency_table = pd.DataFrame()

                for player in players:
                    serve_dir_counts = df_points_selected.loc[(df_points_selected["server_name"].str.contains(player)) & 
                                                              (df_points_selected["serve_side"] == serve_side), serve_seq].value_counts()
        #             print(serve_dir_counts)
                    serve_dir_counts["server_name"] = player
                    contingency_table = contingency_table.append(serve_dir_counts, ignore_index=True)

                contingency_table.set_index("server_name", inplace=True)
                contingency_table = contingency_table[["wide", "body", "t"]]
                chi_square(contingency_table, print_table = True)
                print()
    else:
        print("No match found")

In [26]:
# Compare players' serves when they play each other
compare_player_serves_when_they_play_each_other(df_points, df_matches, "nadal", "federer", "clay")

clay
deuce
Sv1_direction
chi-squared test results:
chi2 test stats = 367.305472
p = 0.000000
dof = 2 
probability=0.950, critical=5.991, stat=367.305
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
              wide   body      t
server_name                     
nadal         78.0  249.0  493.0
federer      443.0   92.0  314.0

Sv2_direction
chi-squared test results:
chi2 test stats = 141.023824
p = 0.000000
dof = 2 
probability=0.950, critical=5.991, stat=141.024
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
              wide  body     t
server_name                   
nadal         24.0  80.0  98.0
federer      202.0  67.0  47.0

ad
Sv1_direction
chi-squared test results:
chi2 test stats = 221.974035
p = 0.000000
dof = 2 
probability=0.950, critical=5.991, stat=221.974
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
              wide   body      t
server_name                     
nadal        628.0   70.0   5

In [27]:
# To-do: Compare players' serves when they play the same player
def compare_player_serves_when_they_play_same_opponent(point_data, match_data, player1, player2, opponent, surface):
    
    my_matches1 = select_matches(data=match_data, player1=player1, player2=opponent, surface=surface)
    if my_matches1.empty:
        print("No match found between " + player1 + " and " + opponent)
    df_points_selected1 = select_players_points(point_data=point_data, match_data=my_matches1)
    
    my_matches2 = select_matches(data=match_data, player1=player2, player2=opponent, surface=surface)
    if my_matches2.empty:
        print("No match found between " + player2 + " and " + opponent)
    df_points_selected2 = select_players_points(point_data=point_data, match_data=my_matches2)
    
    if (df_points_selected1.empty == False) & (df_points_selected2.empty == False):      
        players = [player1, player2]
        serve_sides = ["deuce", "ad"]
        serve_seqs = ["Sv1_direction", "Sv2_direction"]
        surfaces = ["hard", "clay", "grass"]
        
        print(surface)
        for serve_side in serve_sides:
            print(serve_side)
            for serve_seq in serve_seqs:
                print(serve_seq)
                contingency_table = pd.DataFrame()
                
                serve_dir_counts1 = df_points_selected1.loc[(df_points_selected1["server_name"].str.contains(player1)) & 
                                                          (df_points_selected1["serve_side"] == serve_side), serve_seq].value_counts()
                serve_dir_counts1["server_name"] = player1
                contingency_table = contingency_table.append(serve_dir_counts1, ignore_index=True)

                serve_dir_counts2 = df_points_selected2.loc[(df_points_selected2["server_name"].str.contains(player2)) & 
                                                          (df_points_selected2["serve_side"] == serve_side), serve_seq].value_counts()
                serve_dir_counts2["server_name"] = player2
                
                contingency_table = contingency_table.append(serve_dir_counts2, ignore_index=True)
            
                contingency_table.set_index("server_name", inplace=True)
                contingency_table = contingency_table[["wide", "body", "t"]]
                chi_square(contingency_table, print_table = True)
                print()
    else:
        print("No match found")

In [30]:
compare_player_serves_when_they_play_same_opponent(point_data = df_points, match_data = df_matches, player1 = "murray", player2 = "federer", opponent = "djokovic", surface = "hard")

hard
deuce
Sv1_direction
chi-squared test results:
chi2 test stats = 3.476984
p = 0.175785
dof = 2 
probability=0.950, critical=5.991, stat=3.477
Independent (fail to reject H0)
significance=0.050, p=0.176
Independent (fail to reject H0)
              wide   body      t
server_name                     
murray       592.0   90.0  465.0
federer      788.0  148.0  698.0

Sv2_direction
chi-squared test results:
chi2 test stats = 58.799682
p = 0.000000
dof = 2 
probability=0.950, critical=5.991, stat=58.800
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
              wide   body      t
server_name                     
murray        39.0  232.0  187.0
federer      162.0  226.0  213.0

ad
Sv1_direction
chi-squared test results:
chi2 test stats = 51.066264
p = 0.000000
dof = 2 
probability=0.950, critical=5.991, stat=51.066
Dependent (reject H0)
significance=0.050, p=0.000
Dependent (reject H0)
              wide   body      t
server_name                     
murray   

In [125]:
player1 = "djokovic"
player2 = "murray"
tournament = "australian"
date = "2019"

my_matches = select_matches(data=df_matches, player1=player1, player2=player2, tournament=tournament, date=None)

print(my_matches)

                                               match_id        Player 1  \
972   20160131-M-Australian_Open-F-Novak_Djokovic-An...  novak_djokovic   
1256  20150201-M-Australian_Open-F-Novak_Djokovic-An...  novak_djokovic   
1472  20130127-M-Australian_Open-F-Novak_Djokovic-An...  novak_djokovic   
1563  20120127-M-Australian_Open-SF-Novak_Djokovic-A...  novak_djokovic   
1621  20110130-M-Australian_Open-F-Novak_Djokovic-An...  novak_djokovic   

         Player 2 Pl 1 hand Pl 2 hand Gender      Date       Tournament Round  \
972   andy_murray         r         r      M  20160131  australian_open     F   
1256  andy_murray         r         r      M  20150201  australian_open     F   
1472  andy_murray         r         r      M  20130127  australian_open     F   
1563  andy_murray         r         r      M  20120127  australian_open    SF   
1621  andy_murray         r         r      M  20110130  australian_open     F   

     Time            Court Surface        Umpire  Best of Fina

In [135]:
# Check whether a player's serve pattern is consistent within a match. 
# Retrieve all the points each player served, randomly split them into half. 
# Count the frequency of serve directions for each half and conduct a Chi-square test to see they are significantly different, controlled by 
# serve sides and serve sequences.
# We can do the random split multiple times to check the results. 
def check_internal_consistency_split_half(point_data, match_id, player1, player2):
    df_points_selected = df_points.loc[point_data["match_id"] == match_id]
    
    for player in [player1, player2]:
        # Find all the points that each player served.
        print(player)
        df_points_player_served = df_points_selected.loc[df_points_selected["server_name"].str.contains(player), 
                                                          ["serve_side", "server_name", "Sv1_direction", "Sv2_direction"]]

        df_points_player_served_half1 = df_points_player_served.sample(frac=0.5)
        df_points_player_served_half2 = df_points_player_served.drop(df_points_player_served_half1.index)
    #     print(df_points_player1_served_half1)
    #     print(df_points_player1_served_half2)

        for serve_side in ["deuce", "ad"]:
            print(serve_side)
            for serve_seq in ["Sv1_direction", "Sv2_direction"]:

                contingency_table = pd.DataFrame(columns=["group", "wide", "body", "t"])

                print(serve_seq)
                serve_dir_counts1 = df_points_player_served_half1.loc[df_points_player_served_half1["serve_side"] \
                                                                       == serve_side, serve_seq].value_counts(dropna=False)
                serve_dir_counts1.drop(labels = "unknown", inplace=True, errors="ignore")
                serve_dir_counts1["group"] = 1
                contingency_table = contingency_table.append(serve_dir_counts1)
    #             print(serve_dir_counts1)

                serve_dir_counts2 = df_points_player_served_half2.loc[df_points_player_served_half2["serve_side"] \
                                                                       == serve_side, serve_seq].value_counts(dropna=False)
                serve_dir_counts2.drop(labels = "unknown", inplace=True, errors="ignore")
                serve_dir_counts2["group"] = 2
                contingency_table = contingency_table.append(serve_dir_counts2)
    #             print(serve_dir_counts2)

                contingency_table.set_index("group", inplace=True)
                contingency_table.fillna(0, inplace=True)
                print(contingency_table)
                chi_square(contingency_table)
                print()

In [137]:
player1 = "djokovic"
player2 = "murray"
tournament = "australian"
date = "2016"

my_matches = select_matches(data=df_matches, player1=player1, player2=player2, tournament=tournament, date=date)
if len(my_matches) > 0:
    match_id = my_matches["match_id"].values[0]
    check_internal_consistency_split_half(df_points, match_id, player1, player2)
else:
    print("No match found")

djokovic
deuce
Sv1_direction
       wide  body   t
group                
1        10     2  16
2        13     3  13
chi-squared test results:
chi2 test stats = 0.884378
p = 0.642628
dof = 2 
probability=0.950, critical=5.991, stat=0.884
Independent (fail to reject H0)
significance=0.050, p=0.643
Independent (fail to reject H0)

Sv2_direction
       wide  body  t
group               
1         3     5  4
2         5     3  1
chi-squared test results:
chi2 test stats = 2.420833
p = 0.298073
dof = 2 
probability=0.950, critical=5.991, stat=2.421
Independent (fail to reject H0)
significance=0.050, p=0.298
Independent (fail to reject H0)

ad
Sv1_direction
       wide  body   t
group                
1        11     4  13
2        15     4   7
chi-squared test results:
chi2 test stats = 2.344527
p = 0.309665
dof = 2 
probability=0.950, critical=5.991, stat=2.345
Independent (fail to reject H0)
significance=0.050, p=0.310
Independent (fail to reject H0)

Sv2_direction
       wide  body  t
gro

To-do

1. Serve direction analysis
The characteristics of serves for one player
    - Comparison by serve sides(use chi-square)
    - Comparison first and second serves(use chi-square, ind=serves, dep=serve_direction)
    - Comparison by surfaces (control by opponent)(use chi-square)
    - Comparison by opponents (different serve patterns for different opponents?) (use chi-square)
    - Comparison by age (early years vs later years) (chi-square, ind=different ages, dep=serve_directions)
    - Comparison by tournament (grand slam vs other tournaments) (chi-square)
    - Show the change of serve directions during a match (draw a line chart show the counts of each serve direction over time)
    - Serve patterns at critical moments vs non-critical moments (1 points from winning, 1 points from losing, etc.)
    - Serve pattern based on tiredness, group serves by the count of points played (early, middle, or later) and compare between groups.
    - Serve patterns when leading vs when trailing
    - Serve patterns when first serve rate is high (high confidence) vs when the rate is low (low confidence)
    - Serve patterns when winning several points in a row (high confidence), serve patterns when losing several points in a row (low confidence, frustration). 
    - Serve after unforced errors (one or more). Serve after the opponent hitting a winner. Serve after hitting winners. 
    - Serve right after long points. 
    - Serve right after aces (is there a difference? Is he likely to serve a different direction?)
    - Serve right after double faults (is there a difference?)
    - Serve right after repeated first serve faults
    - Is the success or failure of certain serve directions (e.g., aces, short points won, double faults) in the beginning influence the later serve decisions? (anchor effect) Any short term or long term effects?
    - Can we find any subtle bias in serve selection?
    
Compare the serves of two players
    - When they play each other ...
    - When they play the same opponent ...
    - When they are tired
    - When they are leading or losing
    - At critical moments ...
    - At high or low confidence level 

Compare the serves for three or more players?
    - Is it useful?
    
2. Serve error analysis
    - Frequency of errors at critical moments (double fault, first serve fault)
    - Types of errors correlated with tension, confidence, etc. 


In [ ]:
t